# qubits

In [69]:
import cirq

def deutsch_jozsa_oracle(circuit, q0,q1, function_type='balanced'):
    # For a constant function that always returns 0, do nothing (i.e., identity)
    if function_type == 'constant_0':
        return 
    # For a constant function that always returns 1, flip the answer qubit
    elif function_type == 'constant_1':
        return circuit.append(cirq.X(q1))
    # For a balanced function, as an example, you might use a controlled-NOT
    # Here we show a simple controlled-NOT from one of the query qubits to the answer qubit
    elif function_type == 'balanced':
        return circuit.append(cirq.CNOT(q0, q1))
        
def deutsch_jozsa_algo(function):
    # Create a single qubit at position 0 on a 1D line
    qubit0 = cirq.LineQubit(0)
    qubit1 = cirq.LineQubit(1)

    # Create a circuit
    circuit = cirq.Circuit()

    circuit.append([cirq.X(qubit1)])
    circuit.append([cirq.H(qubit0), cirq.H(qubit1)])

    # Append the oracle operations to the circuit
    deutsch_jozsa_oracle(circuit, qubit0, qubit1,function)

    circuit.append([cirq.H(qubit0)])

    print(circuit)
    # Create a simulator
    simulator = cirq.Simulator()

    # Simulate the circuit
    result = simulator.simulate(circuit)
    # Print the state vector in LaTeX format
    print("\nState vector in LaTeX format:")
    print(cirq.dirac_notation(result.final_state_vector, decimals=3))
    
deutsch_jozsa_algo('balanced')
deutsch_jozsa_algo('constant_0')
deutsch_jozsa_algo('constant_1')

# qudits